In [1]:
#Libs
import pandas as pd

#Pré-processamento
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV 

#Automação
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Modelos
from sklearn.ensemble import RandomForestClassifier

#Métricas
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix

In [2]:
#Carregando dataset
df = pd.read_csv('../input/train.csv')

In [3]:
#Variáveis booleanas
bool_vars = ['ind_var30', 'ind_var39_0', 'ind_var43_emit_ult1', 'ind_var43_recib_ult1',
             'ind_var5', 'ind_var5_0']

#Variáveis categóricas
cat_vars = ['num_meses_var12_ult3', 'num_meses_var13_corto_ult3', 'num_meses_var39_vig_ult3',
            'num_meses_var5_ult3', 'num_meses_var8_ult3']

#Variáveis numéricas
num_vars = ['imp_aport_var13_hace3', 'imp_trans_var37_ult1', 'saldo_var30']

In [4]:
## Data sampling
# Função para extrair datasets balanceados
def getBalancedSample(dataFrame, satisfied_ratio = 1, seed = 0):
    #Clientes insatisfeitos
    target1_index = dataFrame[dataFrame.TARGET == 1].index
    
    #Clientes satisfeitos: insatisfeitos * proporção desejada
    satisfied_length = len(target1_index) * satisfied_ratio
    target0_index = dataFrame[dataFrame.TARGET == 0].sample(satisfied_length, random_state = seed).index
    
    return(pd.concat([dataFrame.iloc[target1_index],
                      dataFrame.iloc[target0_index]], axis = 0))

# Datasets para treino
seed = 10
df_11_ratio = getBalancedSample(df, 1, seed)
df_21_ratio = getBalancedSample(df, 2, seed)
df_31_ratio = getBalancedSample(df, 3, seed)

# Dataset para teste
df_test = df.sample(30000, random_state = seed)

# Score
def model_scoring(model, test_data):
    for m in model:
        preds = m.predict(test_data)
        acc = accuracy_score(test_data.TARGET.ravel(), preds)
        rec = recall_score(test_data.TARGET.ravel(), preds)
        roc_auc = roc_auc_score(test_data.TARGET.ravel(), preds)
        cm = confusion_matrix(test_data.TARGET.ravel(), preds)
        print('Accuracy: %.5f\nRecall: %.5f\nROC-AUC: %.5f' % (acc, rec, roc_auc))
        print(cm)
        print('-----')

In [5]:
#Transformers
prep_ct = ColumnTransformer([['encode_cat', OneHotEncoder(categories='auto', handle_unknown='ignore'), cat_vars],
                             ['scale_num', MinMaxScaler(), num_vars],
                             ['pass_bool', 'passthrough', bool_vars]],
                            remainder='drop',)

#Pipeline
def fit_pipe_model(model, dataFrame, seed = 0):
    #Train/valid split
    X_train, X_valid, y_train, y_valid = train_test_split(dataFrame.drop('TARGET', axis=1),
                                                          dataFrame.TARGET.ravel(),
                                                          train_size = 0.7, random_state = seed)
    
    X = pd.concat([dataFrame[bool_vars],
                   dataFrame[cat_vars],
                   dataFrame[num_vars]], axis=1)
    y = dataFrame.TARGET.ravel()
    
    #Processando o pipeline
    pipe = Pipeline(steps=[('Prep', prep_ct),
                           ('Model', model)])
    pipe.fit(X_train, y_train)
    pipe_preds = pipe.predict(X_valid)
    
    print('*** Train Validation ***')
    acc = accuracy_score(y_valid, pipe_preds) 
    rec = recall_score(y_valid, pipe_preds) 
    roc_auc = roc_auc_score(y_valid, pipe_preds)
    cm = confusion_matrix(y_valid, pipe_preds)
    print('Accuracy: %.5f\nRecall: %.5f\nROC-AUC: %.5f' % (acc, rec, roc_auc))
    print(cm)
    print('-----')
    
    return(pipe)

In [6]:
# Treino
rf_model_11 = fit_pipe_model(RandomForestClassifier(n_estimators=10, random_state=10), df_11_ratio, seed)
rf_model_21 = fit_pipe_model(RandomForestClassifier(n_estimators=10, random_state=10), df_21_ratio, seed)
rf_model_31 = fit_pipe_model(RandomForestClassifier(n_estimators=10, random_state=10), df_31_ratio, seed)

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


*** Train Validation ***
Accuracy: 0.67922
Recall: 0.70087
ROC-AUC: 0.67890
[[584 305]
 [274 642]]
-----
*** Train Validation ***
Accuracy: 0.69904
Recall: 0.63123
ROC-AUC: 0.68210
[[1323  482]
 [ 333  570]]
-----
*** Train Validation ***
Accuracy: 0.73380
Recall: 0.08925
ROC-AUC: 0.52335
[[2566  114]
 [ 847   83]]
-----


In [7]:
# Proporção real do dataset de teste
print(df_test.groupby('TARGET')['TARGET'].count())

TARGET
0    28837
1     1163
Name: TARGET, dtype: int64


Dos 30000 registros selecionados para teste, obtivemos 1163 clientes insatisfeitos e 28837 satisfeitos.

Com estes números em mente, vamos avaliar a performance de cada modelo treinado

In [8]:
# Scores
model_scoring([rf_model_11, rf_model_21, rf_model_31], df_test)

Accuracy: 0.65987
Recall: 0.79020
ROC-AUC: 0.72240
[[18877  9960]
 [  244   919]]
-----
Accuracy: 0.29933
Recall: 0.85469
ROC-AUC: 0.56581
[[ 7986 20851]
 [  169   994]]
-----
Accuracy: 0.92743
Recall: 0.21410
ROC-AUC: 0.58515
[[27574  1263]
 [  914   249]]
-----


O modelo 'RF_11' apresentou o melhor resultado, onde conseguiu identificar 919 dos 1163 clientes insatisfeitos com 65.98% de precisão. Vamos utilizar GridSearchCV para otimizar os hiperparâmetros deste modelo enfatizando duas métricas diferentes:

* Accuracy (ênfase em classificar corretamente clientes satisfeitos e insatisfeitos)
* Recall (ênfase em identificar principalmente os clientes insatisfeitos)

In [9]:
#Hiperparâmetros a testar
rf_params = {'n_estimators': [10, 30, 100],
             'max_depth': [2, 4, 6, None],
             'min_samples_split': [2, 10, 20, 30],
             'min_samples_leaf': [1, 5, 10, 50]}

#Novo modelo
rf_cv_acc = GridSearchCV(RandomForestClassifier(random_state=10), param_grid = rf_params,
                         scoring = 'accuracy', cv = 5, n_jobs = -1)

rf_cv_rec = GridSearchCV(RandomForestClassifier(random_state=10), param_grid = rf_params,
                         scoring = 'recall', cv = 5, n_jobs = -1)

#Treino do modelo
rf_model_11_cv_acc = fit_pipe_model(rf_cv_acc, df_11_ratio, seed)
rf_model_11_cv_rec = fit_pipe_model(rf_cv_rec, df_11_ratio, seed)

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


*** Train Validation ***
Accuracy: 0.69197
Recall: 0.69869
ROC-AUC: 0.69186
[[609 280]
 [276 640]]
-----


/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


*** Train Validation ***
Accuracy: 0.68199
Recall: 0.70524
ROC-AUC: 0.68164
[[585 304]
 [270 646]]
-----


In [10]:
#Score
model_scoring([rf_model_11_cv_acc, rf_model_11_cv_rec], df_test)

Accuracy: 0.68177
Recall: 0.76268
ROC-AUC: 0.72059
[[19566  9271]
 [  276   887]]
-----
Accuracy: 0.66313
Recall: 0.81857
ROC-AUC: 0.73772
[[18942  9895]
 [  211   952]]
-----


Para o modelo com ênfase em Accuracy, obtivemos um ganho de 3% nesta métrica ao custo de 3% em recall. Observando a ConfusionMatrix, o modelo melhorou a classificação de clientes satisfeitos, ao passo que piorou a classificação de clientes insatisfeitos

Para o modelo com ênfase em Recall, observamos um ganho de 1% em Accuracy e 3% nesta métrica. Conseguimos identificar corretamente mais clientes satisfeitos e insatisfeitos.

Vamos submeter os três modelos ao Kaggle e avaliar a performance na competição

In [11]:
#Realizando previsões com os dados de teste da competição
df_submission = pd.read_csv('../input/test.csv')

rf_11_preds = rf_model_11.predict(df_submission)
rf_11_cv_acc_preds = rf_model_11_cv_acc.predict(df_submission)
rf_11_cv_rec_preds = rf_model_11_cv_rec.predict(df_submission)

In [12]:
#Preparando arquivo para submission
rf_11_submission = pd.read_csv('../input/sample_submission.csv')
rf_11_cv_acc_submission = pd.read_csv('../input/sample_submission.csv')
rf_11_cv_rec_submission = pd.read_csv('../input/sample_submission.csv')


rf_11_submission.TARGET = rf_11_preds
rf_11_cv_acc_submission.TARGET = rf_11_cv_acc_preds
rf_11_cv_rec_submission.TARGET = rf_11_cv_rec_preds

In [13]:
rf_11_submission.to_csv('../output/rf_11_submission.csv', index=False)
rf_11_cv_acc_submission.to_csv('../output/rf_11_cv_acc_submission.csv', index=False)
rf_11_cv_rec_submission.to_csv('../output/rf_11_cv_rec_submission.csv', index=False)

![Kaggle_submission](../presentation/kaggle_submission.png)

A métrica utilizada na competição é a ROC-AUC. Os resultados obtidos estão ligeiramente inferiores aos observados durante a criação do modelo neste trabalho.

Os três modelos apresentaram performances similares, sendo que o 'RF_11_cv_acc' apresentou o melhor resultado.

In [14]:
best_model = rf_model_11_cv_acc.named_steps['Model'].best_estimator_

In [15]:
#Hiperparâmetros selecionados para o modelo
best_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

Dentre os parâmetros fornecidos, a melhor combinação encontrada foi:

* n_estimators=100
* max_depth=None
* min_samples_leaf=1
* min_samples_split=20

As árvores de decisão realizaram splits a cada 20 amostras. Entretanto, cada amostra resultou na criação de um leaf (decisão), o que sugere baixa capacidade de generalização deste modelo visto que cada amostra ao final de um split resultará em uma decisão.

In [16]:
#Observando o grau de importância das features selecionadas
pd.Series(best_model.feature_importances_)

0     0.011071
1     0.001972
2     0.003319
3     0.006224
4     0.010221
5     0.001313
6     0.003224
7     0.002525
8     0.008605
9     0.026395
10    0.014645
11    0.003176
12    0.079455
13    0.011542
14    0.015184
15    0.098420
16    0.014511
17    0.002993
18    0.004351
19    0.000880
20    0.008614
21    0.045503
22    0.429923
23    0.066695
24    0.014161
25    0.019397
26    0.012211
27    0.067256
28    0.016215
dtype: float64

Os indices acima representam:

* 0 a 19: Variáveis categóricas processadas com OneHotEncoding
* 20 a 22: Variáveis numéricas selecionadas via PCA
* 23 a 28: Variáveis booleanas selecionadas

A maioria das features utilizadas neste modelo possui baixa importância. sendo a variável 22 (saldo_var30) a mais significativa.

Essa análise sugere que podemos melhorar a performance deste modelo selecionando e/ou concebendo novas features.